In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib 
import gc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# -----------------------------
# Download and preprocess data
# -----------------------------
ticker = "AAPL"
df = yf.download(ticker, start="2015-01-01", end="2023-12-31").reset_index()
df['return'] = df['Close'].pct_change()
df['daily_range'] = df['High'] - df['Low']
df['close_open_diff'] = df['Close'] - df['Open']

# Fill NAs created by pct_change
df.fillna(0, inplace=True)

# Features
features = ['Close', 'return', 'daily_range', 'close_open_diff', 'Volume']

# Scale features
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[features])

# -----------------------------
# Create sequences for LSTM
# -----------------------------
def create_sequences(data, window=10):
    X, y = [], []
    for i in range(window, len(data)-1):
        X.append(data[i-window:i])      # past 'window' days
        y.append(data[i, 0])            # next-day Close price
    return np.array(X), np.array(y)

lookback = 20
X, y = create_sequences(scaled_data, window=lookback)

# Train-test split (80-20)
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# -----------------------------
# Build LSTM model
# -----------------------------
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# Predict
y_pred = model.predict(X_test)

# Reverse scaling for Close price
y_test_rescaled = y_test * scaler.scale_[0] + scaler.min_[0]
y_pred_rescaled = y_pred.flatten() * scaler.scale_[0] + scaler.min_[0]

# Evaluate
from sklearn.metrics import mean_squared_error, r2_score
rmse = np.sqrt(mean_squared_error(y_test_rescaled, y_pred_rescaled))
r2 = r2_score(y_test_rescaled, y_pred_rescaled)
print(f'RMSE: {rmse:.4f}, R2: {r2:.4f}')

model.save("lstm_stock_model.keras")
joblib.dump(scaler, "scaler.pkl")

print("lstm_stock_model.keras")



# -----------------------------
# 16. Cleanup
# -----------------------------
del model
gc.collect()










/tmp/ipykernel_19209/1028500230.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2015-01-01", end="2023-12-31").reset_index()
[*********************100%***********************]  1 of 1 completed

Epoch 1/100



/home/parbhat/Desktop/AiMl/venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.0105 - val_loss: 0.0029
Epoch 2/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0027 - val_loss: 0.0022
Epoch 3/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0020 - val_loss: 0.0016
Epoch 4/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0018 - val_loss: 0.0017
Epoch 5/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0016 - val_loss: 0.0038
Epoch 6/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0015 - val_loss: 0.0016
Epoch 7/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0014 - val_loss: 0.0012
Epoch 8/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0015 - val_loss: 0.0011
Epoch 9/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0016 - val_loss: 0.0012
Epoch 10/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0014 - val_loss: 9.8334e-04
Epoch 11/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0015 - val_loss: 0.0021
Epoch 12/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss:

11691